In [2]:
from pymongo import MongoClient
import pandas as pd

# Connexion MongoDB
client = MongoClient("localhost", 27017)
db = client["IF29_twitter_db"]

# Charger les outliers K-Means (acp_outliers)
acp_outliers_df = pd.DataFrame(list(db["acp_outliers"].find({}, {"_id": 1})))
acp_mapping_df = pd.DataFrame(list(db["acp_db"].find({}, {"_id": 1, "user_id": 1})))
acp_outliers_with_uid = pd.merge(acp_outliers_df, acp_mapping_df, on="_id", how="left")
acp_user_ids = set(acp_outliers_with_uid["user_id"].astype(str))

# Charger les outliers Random Forest (supervisé)
rf_df = pd.DataFrame(list(db["users_labeled_outliers_supervised"].find({"is_outlier": 1}, {"_id": 1})))
rf_user_ids = set(rf_df["_id"].astype(str))

# Comparaison
common_user_ids = acp_user_ids & rf_user_ids

# Résumé
summary = {
    "KMeans_outliers": len(acp_user_ids),
    "RF_outliers": len(rf_user_ids),
    "Common_outliers": len(common_user_ids),
    "Pourcentage_RF_dans_KMeans": round(len(common_user_ids) / len(rf_user_ids) * 100, 2) if rf_user_ids else 0,
    "Pourcentage_KMeans_dans_RF": round(len(common_user_ids) / len(acp_user_ids) * 100, 2) if acp_user_ids else 0
}

summary_df = pd.DataFrame([summary])
print("\n=== Comparaison des outliers détectés (KMeans vs Random Forest) ===\n")
print(summary_df.to_string(index=False))



=== Comparaison des outliers détectés (KMeans vs Random Forest) ===

 KMeans_outliers  RF_outliers  Common_outliers  Pourcentage_RF_dans_KMeans  Pourcentage_KMeans_dans_RF
            2676         2675             1299                       48.56                       48.54
